# P.1 Intro and Neuron Code
Neural networks comprise of layers of neurons and connections between the neurons of each layer. Tuning the weights and biases of these connections allow the network to "learn" and predict.

# Neuron code
Suppose we are looking at a single neuron taking in 3 inputs from the previous layer. The following is a simplified look at what a neuron does. It takes a weighted sum of its inputs and adds the bias associated with the neuron. The result of this calculation is output of the neuron.

In [3]:
inputs = [1, 2, 3] # Output of previous layer's neurons
weights = [0.2, 0.8, -0.5] # Strength of connection between the previous layer's neurons
bias = 2 # Bias associated with this particular neuron

output = inputs[0]*weights[0] + inputs[1]*weights[1] + inputs[2]*weights[2] + bias
print(output)

2.3
